In [1]:
import geopandas as gpd
import pandas as pd
import rasterio
import boto3
import os

In [2]:
download_dir = './downloads/'
image_files = os.listdir(download_dir)

In [3]:
bucket_name = 'opera-calval-database-dswx'

In [4]:
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')
s3_client = session.client('s3')

In [5]:
obj = s3.Object(bucket_name,'site.geojson')
site_table_data = obj.get()['Body']
sites = gpd.read_file(site_table_data)
sites.head()

,site_name,water_content,geometry
0,3_42,3.0,"POLYGON ((-122.96079 57.91963, -122.77746 57.9..."
1,4_21,4.0,"POLYGON ((-72.05905 55.21964, -71.88820 55.219..."
2,2_8,2.0,"POLYGON ((-70.87737 -52.06050, -70.71788 -52.0..."
3,4_3,4.0,"POLYGON ((33.66289 -16.24069, 33.66287 -16.339..."
4,4_34,4.0,"POLYGON ((-96.71059 62.77966, -96.57702 62.779..."


In [6]:
obj = s3.Object(bucket_name,'image.geojson')
image_table_data = obj.get()['Body']
images = gpd.read_file(image_table_data)
images.head()

,cloud_cover,collocated_dswx,datetime,image_name,instrument,provider,resolution,s3_bucket,s3_key_image,site_coverage,site_name,timeDelta_days,geometry
0,NaN,None,None,None,None,None,NaN,None,None,NaN,None,NaN,None
1,NaN,None,None,None,None,None,NaN,None,None,NaN,None,NaN,None
2,NaN,None,None,None,None,None,NaN,None,None,NaN,None,NaN,None
3,NaN,None,None,None,None,None,NaN,None,None,NaN,None,NaN,None
4,NaN,None,None,None,None,None,NaN,None,None,NaN,None,NaN,None


In [7]:
sites.head()

,site_name,water_content,geometry
0,3_42,3.0,"POLYGON ((-122.96079 57.91963, -122.77746 57.9..."
1,4_21,4.0,"POLYGON ((-72.05905 55.21964, -71.88820 55.219..."
2,2_8,2.0,"POLYGON ((-70.87737 -52.06050, -70.71788 -52.0..."
3,4_3,4.0,"POLYGON ((33.66289 -16.24069, 33.66287 -16.339..."
4,4_34,4.0,"POLYGON ((-96.71059 62.77966, -96.57702 62.779..."


In [8]:
newSites = gpd.GeoDataFrame(columns=['geometry'], geometry='geometry')
for file in image_files:
    if file[-3:] != 'tif':
        continue
    
    ID = file.split('_3B_Analytic')[0]
    #print(ID)
    imagery = rasterio.open(download_dir+file)
    image_crs = imagery.crs
    chip_ID = images[images.image_name == ID].site_name.iloc[0]
    chip = sites[sites.site_name == chip_ID]
    chip['geometry'] = chip.geometry.to_crs(image_crs).buffer(-500,join_style=2).to_crs(epsg=4326)
    
    newSites = newSites.append(chip)

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [9]:
newSites.head()

,geometry,site_name,water_content
0,"POLYGON ((-122.95236 57.91514, -122.78589 57.9...",3_42,3.0
1,"POLYGON ((-72.05120 55.21516, -71.89605 55.215...",4_21,4.0
2,"POLYGON ((-70.87008 -52.06500, -70.72517 -52.0...",2_8,2.0
3,"POLYGON ((33.56456 -16.33480, 33.56454 -16.245...",4_3,4.0
4,"POLYGON ((-96.70081 62.77517, -96.57702 62.775...",4_34,4.0


In [10]:
newSites.to_file('newSites.geojson', driver="GeoJSON")  

In [11]:
newSites_bytes = bytes(newSites.to_json(drop_id=True).encode('UTF-8'))
s3object = s3.Object(bucket_name,'site.geojson')
s3object.put(Body=newSites_bytes)

{'ResponseMetadata': {'RequestId': 'S9P382EKKHBAZ75J',
  'HostId': 'tdoKs+nIj8xvphrj5CaddV5rlGCR2GTUFOvAK38JesY7A/S0aGOmajslgDGlr3U9OUlmlQUj2sk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'tdoKs+nIj8xvphrj5CaddV5rlGCR2GTUFOvAK38JesY7A/S0aGOmajslgDGlr3U9OUlmlQUj2sk=',
   'x-amz-request-id': 'S9P382EKKHBAZ75J',
   'date': 'Mon, 01 Aug 2022 22:46:22 GMT',
   'etag': '"2f54a7e41ad302e0aba45c0ee4af08ba"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"2f54a7e41ad302e0aba45c0ee4af08ba"'}